<ul class="breadcrumb">
  <li><a href="2.1.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="2.3.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font><br> 
  <font size="5" face="Georgia">(with Maternal Effects)</font> 

</div>

In [2]:
using DataFrames
using JWAS: MT,Datasets

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multivariate Linear Additive Genetic Model</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [3]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [4]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [6]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [7]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [8]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [9]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [10]:
model    = MT.build_model(model_equations,R);

In [11]:
MT.set_covariate(model,"age")

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [12]:
ped = MT.get_pedigree(pedfile);

Finished!


In [13]:
MT.set_random(model,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [14]:
out2 = MT.runMCMC(model,data,chain_length=1000,printout_frequency=500,output_samples_frequency=10);

running MCMC for conventional (no markers)...  0%|      |  ETA: 0:04:48
Posterior means at iteration: 500
Residual covariance matrix: 
[25.281 5.213
 5.213 1.655]
Polygenic effects covariance matrix 
[146.853 32.91
 32.91 9.769]

running MCMC for conventional (no markers)... 50%|███   |  ETA: 0:00:01
Posterior means at iteration: 1000
Residual covariance matrix: 
[74.781 16.031
 16.031 4.19]
Polygenic effects covariance matrix 
[85.494 17.066
 17.066 6.098]

running MCMC for conventional (no markers)...100%|██████| Time: 0:00:01


In [15]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 5 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Posterior mean of residual covariance matrix"
  "MCMC samples for residual covariance matrix"
  "MCMC samples for polygenic effects covariance matrix"
  "Posterior mean of location parameters"

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. The Multivariate Linear Additive Genetic Model with Maternal Effects</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [16]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [17]:
d2 = [data[3:4,:]  DataFrame(Dam = ["D1","D1"])]

,Animal,BW,CW,age,sex,Dam
1,O1,150.0,13.0,3,M,D1
2,O3,40.0,5.0,4,F,D1


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [18]:
model_equations = "BW = intercept + age + sex + Animal+ Dam;
                   CW = intercept + age + sex + Animal";

In [19]:
model = MT.build_model(model_equations,R);

In [20]:
MT.set_covariate(model,"age")

In [21]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
MT.set_random(model,"Animal Dam", ped,G0)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [22]:
out5=MT.runMCMC(model,d2,chain_length=1000,printout_frequency=500);


Posterior means at iteration: 500
Residual covariance matrix: 
[12.27 2.241
 2.241 0.983]
Polygenic effects covariance matrix 
[5.47 1.162 0.38
 1.162 1.41 -0.059
 0.38 -0.059 0.593]

running MCMC for conventional (no markers)... 52%|███   |  ETA: 0:00:00
Posterior means at iteration: 1000
Residual covariance matrix: 
[13.621 3.499
 3.499 1.701]
Polygenic effects covariance matrix 
[5.429 0.928 0.303
 0.928 1.111 -0.042
 0.303 -0.042 0.548]

running MCMC for conventional (no markers)...100%|██████| Time: 0:00:00
